![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [1]:
# Import pandas
import pandas as pd

# Read CSV file
ds_jobs = pd.read_csv("customer_train.csv")

# Check original DataFrame
print(ds_jobs.info())
print(ds_jobs.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [2]:
# Change integer and float columns
ds_jobs[["student_id", "training_hours", "job_change"]] = ds_jobs[["student_id", "training_hours", "job_change"]].astype("int32")
ds_jobs["city_development_index"] = ds_jobs["city_development_index"].astype("float16")
print(ds_jobs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int32  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float16
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int32  
 13  job_change              19158 non-null  int32  
dtypes: float16(1), int32(3), object(10)
me

In [3]:
# Change nominal categories
nom_categories = ["city",
                 "gender",
                 "major_discipline",
                 "company_type"
                ]

for col in nom_categories:
    ds_jobs[col] = ds_jobs[col].astype("category")

In [4]:
print(ds_jobs["last_new_job"].unique())

['1' '>4' 'never' '4' '3' '2' nan]


In [5]:
# Change ordinal categories
order_education_level = ["Primary School", "High School", "Graduate", "Masters", "Phd"]
order_company_size = ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"]
order_relevant_experience = ["No relevant experience", "Has relevant experience"]
order_enrolled_university = ["no_enrollment", "Part time course", "Full time course"]
order_last_job = ["never", "1", "2", "3", "4", ">4"]

education_level = pd.CategoricalDtype(order_education_level, ordered=True)
company_size = pd.CategoricalDtype(order_company_size, ordered=True)
relevant_experience = pd.CategoricalDtype(order_relevant_experience, ordered=True)
enrolled_university = pd.CategoricalDtype(order_enrolled_university, ordered=True)
last_job = pd.CategoricalDtype(order_last_job, ordered=True)

ds_jobs["education_level"] = ds_jobs["education_level"].astype(education_level)
ds_jobs["company_size"] = ds_jobs["company_size"].astype(company_size)
ds_jobs["relevant_experience"] = ds_jobs["relevant_experience"].astype(relevant_experience)
ds_jobs["enrolled_university"] = ds_jobs["enrolled_university"].astype(enrolled_university)
ds_jobs["last_new_job"] = ds_jobs["last_new_job"].astype(last_job)

# Work experience values
list_twenty = list(ds_jobs["experience"].dropna().unique())
list_twenty.remove(">20")
list_twenty.remove("<1")
list_twenty = [int(item) for item in list_twenty]
list_twenty.sort()
list_twenty = [str(item) for item in list_twenty]
list_twenty = ["<1"] + list_twenty + [">20"]

twenty = pd.CategoricalDtype(list_twenty, ordered=True)
ds_jobs["experience"] = ds_jobs["experience"].astype(twenty)

In [6]:
# Copying the clean dataset
ds_jobs_clean = ds_jobs

# Sub-setting for companies between 1,000 and 4,999 employees with 10+ years of exp
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean["experience"] >= "10") & (ds_jobs_clean["company_size"] >= "1000-4999")]
print(ds_jobs_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1